In [ ]:
import cv2
import numpy as np
from pycocotools.coco import COCO
import os
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from matplotlib.patches import Rectangle
from PIL import Image
import random
import json

In [ ]:
# ================== TODO ===========================
initial = 'cje'
read_train_json = 'train_'+initial+'_b.json'                # 읽을 train json 파일 - battey + cutmix
read_train_json_mosaic = 'train_'+initial+'_b_m.json'       # 읽을 train json 파일 - battery + mosaic + cutmix
write_train_json = 'train_'+initial+'_b_c.json'             # 쓸 train json 파일 - battey + cutmix
write_train_json_mosaic = 'train_'+initial+'_b_m_c.json'    # 쓸 train json 파일 - battery + mosaic + cutmix

read_val_json = 'val_'+initial+'_b.json'                    # 읽을 val json 파일 - battey + cutmix
read_val_json_mosaic = 'val_'+initial+'_b_m.json'           # 읽을 val json 파일 - battery + mosaic + cutmix
write_val_json = 'val_'+initial+'_b_c.json'                 # 쓸 val json 파일 - battey + cutmix
write_val_json_mosaic = 'val_'+initial+'_b_m_c.json'        # 쓸 val json 파일 - battery + mosaic + cutmix
folder_dir = 'battery_cutmix_'+initial                      # 이미지 파일을 저장할 폴더명
# ===================================================
# Set the output directory for augmented images
if not os.path.exists(folder_dir):
    os.makedirs(folder_dir)

In [ ]:
iou_t = 50

In [ ]:
# Calculate the Intersection over Union (IoU) between two bounding boxes
def calculate_iou(bbox1, bbox2):
    x1, y1, w1, h1 = bbox1
    x2, y2, w2, h2 = bbox2

    # Calculate coordinates of the intersection rectangle
    x_intersection = max(x1, x2)
    y_intersection = max(y1, y2)
    w_intersection = max(0, min(x1 + w1, x2 + w2) - x_intersection)
    h_intersection = max(0, min(y1 + h1, y2 + h2) - y_intersection)

    # Calculate areas of intersection and union
    intersection_area = w_intersection * h_intersection
    bbox1_area = w1 * h1
    bbox2_area = w2 * h2
    union_area = bbox1_area + bbox2_area - intersection_area

    # Calculate IoU
    iou = intersection_area / union_area if union_area > 0 else 0

    return iou

In [ ]:
LABEL_NAME = [
        "General trash", "Paper", "Paper pack", "Metal", "Glass",
        "Plastic", "Styrofoam", "Plastic bag", "Battery", "Clothing"
    ]

## Train dataset Up-sampling - CutMix

In [ ]:
# cutmix train - 200000 부터 시작
new_image_id = 200000
new_annotation_id = 200000
coco_train = COCO(read_train_json)

In [ ]:
source_image_ids = [img['id'] for img in coco_train.dataset['images']]
num_augmented_images = len(source_image_ids)
print(num_augmented_images)

In [ ]:
# Initialize the new images and annotations lists
new_images = []
new_annotations = []
target_labels = ["Styrofoam", "Glass", "Metal", "Paper pack", "Clothing", "Battery"]

In [ ]:
# Iterate through the augmented images and add their details to the 'images' field
for i in range(num_augmented_images):
    # Select a source image in order
    source_image_id = source_image_ids[i]
    source_image_info = coco_train.imgs[source_image_id]
    source_image_path = source_image_info['file_name']
    source_image = Image.open(source_image_path)
    source_image_width, source_image_height = source_image.size

    # Retrieve the bounding boxes and labels for the source image
    source_ann_ids = coco_train.getAnnIds(imgIds=source_image_info['id'])
    source_annotations = coco_train.loadAnns(source_ann_ids)

    target_annotations = [
        ann for ann in source_annotations if LABEL_NAME[ann['category_id']] in target_labels
    ]

    if len(target_annotations) == 0:
        continue

    # Randomly select a target bounding box from the source image
    target_annotation = random.choice(target_annotations)
    target_bbox = target_annotation['bbox']
    target_label = LABEL_NAME[target_annotation['category_id']]

    # Keep selecting a destination image until one is found that only contains the desired classes
    while True:
        destination_image_id = random.choice(source_image_ids)
        destination_image_info = coco_train.imgs[destination_image_id]
        destination_ann_ids = coco_train.getAnnIds(imgIds=destination_image_info['id'])
        destination_annotations = coco_train.loadAnns(destination_ann_ids)

        # Check if all annotations belong to the desired classes
        if all(LABEL_NAME[ann['category_id']] in target_labels for ann in destination_annotations):
            break  # Exit the loop if a suitable image is found

    destination_image_path = destination_image_info['file_name']
    destination_image = Image.open(destination_image_path)
    destination_image_width, destination_image_height = destination_image.size

    # Retrieve the existing annotations for the destination image
    destination_ann_ids = coco_train.getAnnIds(imgIds=destination_image_info['id'])
    destination_annotations = coco_train.loadAnns(destination_ann_ids)

    # Create a new image for augmentation by pasting the target bounding box
    augmented_image = destination_image.copy()

    # Calculate the position and size for the pasted bounding box
    target_x, target_y, target_w, target_h = target_bbox
    target_x1 = int(target_x)
    target_y1 = int(target_y)
    target_x2 = int(target_x + target_w)
    target_y2 = int(target_y + target_h)

    # Find a suitable position to paste the target bounding box
    max_trials = 50
    trial_count = 0

    while trial_count < max_trials:
        # Generate random coordinates for the paste position
        paste_x = random.randint(0, destination_image_width - int(target_w))
        paste_y = random.randint(0, destination_image_height - int(target_h))

        # Check if the pasted bounding box overlaps with existing bounding boxes
        paste_bbox = [paste_x, paste_y, paste_x + int(target_w), paste_y + int(target_h)]
        overlapping_bboxes = [
            ann for ann in destination_annotations if calculate_iou(ann['bbox'], paste_bbox) > iou_t
        ]

        # If there are no overlapping bounding boxes, proceed with the paste operation
        if len(overlapping_bboxes) == 0:
            augmented_image.paste(
                source_image.crop((target_x1, target_y1, target_x2, target_y2)), (paste_x, paste_y)
            )
            annotation_bbox = [paste_x, paste_y, target_w, target_h]

            # Update the destination annotations with the pasted bounding box
            new_annotation = {
                'id': new_annotation_id,
                'image_id': int(new_image_id),
                'category_id': LABEL_NAME.index(target_label),
                'bbox': annotation_bbox,
                'area': target_w * target_h,
                'iscrowd': 0
            }
            new_annotation_id += 1

            coco_train.dataset['annotations'].append(new_annotation)

            # Update the destination annotations list
            destination_annotations.append(new_annotation)

            break  # Exit the loop once a successful paste is done

        trial_count += 1

    # Save the augmented image
    augmented_image.save(folder_dir+'/'+str(new_image_id)+'.jpg')

    # Add the augmented image details to the COCO dataset
    new_image_info = {
        'id': int(new_image_id),
        'file_name': folder_dir+'/'+str(new_image_id)+'.jpg',
        'width': destination_image_width,
        'height': destination_image_height
    }
    coco_train.dataset['images'].append(new_image_info)

    # Update the image_id in these annotations to match the new image
    for annotation in destination_annotations:
        # Only copy the annotation if it's not the new one
        if annotation['id'] != new_annotation_id - 1:
            annotation_copy = annotation.copy()
            annotation_copy['id'] = new_annotation_id
            annotation_copy['image_id'] = new_image_id
            new_annotations.append(annotation_copy)

            # Increment the annotation ID after creating a new annotation
            new_annotation_id += 1

    # Append the new_annotation to the new_annotations list
    new_annotations.append(new_annotation)

    # Append the new image info to the new_images list
    new_images.append(new_image_info)

    # Increment the image ID and successful paste counter
    new_image_id += 1


In [ ]:
# Extend the original dataset with the new lists
with open(read_train_json, 'r') as f:
    original_dataset = json.load(f)
original_dataset['annotations'].extend(new_annotations)
original_dataset['images'].extend(new_images)

# Save the updated annotations to a new JSON file
with open(write_train_json, 'w') as outfile:
    json.dump(original_dataset, outfile)
    
with open(read_train_json_mosaic, 'r') as f:
    original_dataset = json.load(f)
original_dataset['annotations'].extend(new_annotations)
original_dataset['images'].extend(new_images)

# Save the updated annotations to a new JSON file
with open(write_train_json_mosaic, 'w') as outfile:
    json.dump(original_dataset, outfile)

## Validation dataset Up-sampling - CutMix

In [ ]:
# cutmix train - 200000 부터 시작
new_image_id = 210000
new_annotation_id = 210000
coco_val = COCO(read_val_json)

In [ ]:
source_image_ids = [img['id'] for img in coco_val.dataset['images']]
num_augmented_images = len(source_image_ids)
print(num_augmented_images)

In [ ]:
# Initialize the new images and annotations lists
new_images = []
new_annotations = []
target_labels = ["Styrofoam", "Glass", "Metal", "Paper pack", "Clothing", "Battery"]

In [ ]:
# Iterate through the augmented images and add their details to the 'images' field
for i in range(num_augmented_images):
    # Select a source image in order
    source_image_id = source_image_ids[i]
    source_image_info = coco_val.imgs[source_image_id]
    source_image_path = source_image_info['file_name']
    source_image = Image.open(source_image_path)
    source_image_width, source_image_height = source_image.size

    # Retrieve the bounding boxes and labels for the source image
    source_ann_ids = coco_val.getAnnIds(imgIds=source_image_info['id'])
    source_annotations = coco_val.loadAnns(source_ann_ids)

    target_annotations = [
        ann for ann in source_annotations if LABEL_NAME[ann['category_id']] in target_labels
    ]

    if len(target_annotations) == 0:
        continue

    # Randomly select a target bounding box from the source image
    target_annotation = random.choice(target_annotations)
    target_bbox = target_annotation['bbox']
    target_label = LABEL_NAME[target_annotation['category_id']]

    # Keep selecting a destination image until one is found that only contains the desired classes
    while True:
        destination_image_id = random.choice(source_image_ids)
        destination_image_info = coco_val.imgs[destination_image_id]
        destination_ann_ids = coco_val.getAnnIds(imgIds=destination_image_info['id'])
        destination_annotations = coco_val.loadAnns(destination_ann_ids)

        # Check if all annotations belong to the desired classes
        if all(LABEL_NAME[ann['category_id']] in target_labels for ann in destination_annotations):
            break  # Exit the loop if a suitable image is found

    destination_image_path = destination_image_info['file_name']
    destination_image = Image.open(destination_image_path)
    destination_image_width, destination_image_height = destination_image.size

    # Retrieve the existing annotations for the destination image
    destination_ann_ids = coco_val.getAnnIds(imgIds=destination_image_info['id'])
    destination_annotations = coco_val.loadAnns(destination_ann_ids)

    # Create a new image for augmentation by pasting the target bounding box
    augmented_image = destination_image.copy()

    # Calculate the position and size for the pasted bounding box
    target_x, target_y, target_w, target_h = target_bbox
    target_x1 = int(target_x)
    target_y1 = int(target_y)
    target_x2 = int(target_x + target_w)
    target_y2 = int(target_y + target_h)

    # Find a suitable position to paste the target bounding box
    max_trials = 50
    trial_count = 0

    while trial_count < max_trials:
        # Generate random coordinates for the paste position
        paste_x = random.randint(0, destination_image_width - int(target_w))
        paste_y = random.randint(0, destination_image_height - int(target_h))

        # Check if the pasted bounding box overlaps with existing bounding boxes
        paste_bbox = [paste_x, paste_y, paste_x + int(target_w), paste_y + int(target_h)]
        overlapping_bboxes = [
            ann for ann in destination_annotations if calculate_iou(ann['bbox'], paste_bbox) > iou_t
        ]

        # If there are no overlapping bounding boxes, proceed with the paste operation
        if len(overlapping_bboxes) == 0:
            augmented_image.paste(
                source_image.crop((target_x1, target_y1, target_x2, target_y2)), (paste_x, paste_y)
            )
            annotation_bbox = [paste_x, paste_y, target_w, target_h]

            # Update the destination annotations with the pasted bounding box
            new_annotation = {
                'id': new_annotation_id,
                'image_id': int(new_image_id),
                'category_id': LABEL_NAME.index(target_label),
                'bbox': annotation_bbox,
                'area': target_w * target_h,
                'iscrowd': 0
            }
            new_annotation_id += 1

            coco_val.dataset['annotations'].append(new_annotation)

            # Update the destination annotations list
            destination_annotations.append(new_annotation)

            break  # Exit the loop once a successful paste is done

        trial_count += 1

    # Save the augmented image
    augmented_image.save(folder_dir+'/'+str(new_image_id)+'.jpg')

    # Add the augmented image details to the COCO dataset
    new_image_info = {
        'id': int(new_image_id),
        'file_name': folder_dir+'/'+str(new_image_id)+'.jpg',
        'width': destination_image_width,
        'height': destination_image_height
    }
    coco_val.dataset['images'].append(new_image_info)

    # Update the image_id in these annotations to match the new image
    for annotation in destination_annotations:
        # Only copy the annotation if it's not the new one
        if annotation['id'] != new_annotation_id - 1:
            annotation_copy = annotation.copy()
            annotation_copy['id'] = new_annotation_id
            annotation_copy['image_id'] = new_image_id
            new_annotations.append(annotation_copy)

            # Increment the annotation ID after creating a new annotation
            new_annotation_id += 1

    # Append the new_annotation to the new_annotations list
    new_annotations.append(new_annotation)

    # Append the new image info to the new_images list
    new_images.append(new_image_info)

    # Increment the image ID and successful paste counter
    new_image_id += 1


In [ ]:
# Extend the original dataset with the new lists
with open(read_val_json, 'r') as f:
    original_dataset = json.load(f)
original_dataset['annotations'].extend(new_annotations)
original_dataset['images'].extend(new_images)

# Save the updated annotations to a new JSON file
with open(write_val_json, 'w') as outfile:
    json.dump(original_dataset, outfile)
    
with open(read_val_json_mosaic, 'r') as f:
    original_dataset = json.load(f)
original_dataset['annotations'].extend(new_annotations)
original_dataset['images'].extend(new_images)

# Save the updated annotations to a new JSON file
with open(write_val_json_mosaic, 'w') as outfile:
    json.dump(original_dataset, outfile)